In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from classic_network import LeNet_5, AlexNet, VGG16
%matplotlib inline

# LeNet-5

In [2]:
tf.reset_default_graph()

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.mean(axis=-1)
x_test = x_test.mean(axis=-1)

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 1])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 10)

In [5]:
LR = 5e-4
logits = LeNet_5(x)
cost = tf.losses.mean_squared_error(one_hot_y, logits)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LR)
# TODO, learning rate decay
training = optimizer.minimize(cost)

In [6]:
# TODO Run model and evaluate

# AlexNet

In [7]:
tf.reset_default_graph()

In [8]:
# TODO: Download ImageNet

In [9]:
x = tf.placeholder(tf.float32, shape=[None, 227, 227, 3])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 1000)

In [10]:
BS = 128
Mom = 0.9
LR = 0.0005
logits = AlexNet(x)
cost = tf.losses.softmax_cross_entropy(one_hot_y, logits)
optimizer = tf.train.MomentumOptimizer(learning_rate=LR,
                                       momentum=Mom)
training = optimizer.minimize(cost)

# VGG16

In [11]:
tf.reset_default_graph()

In [12]:
# TODO: Crop ImageNet

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 1000)

In [14]:
BS = 256
Mom = 0.9
LR = 1e-2
logits = VGG16(x)
cost = tf.losses.softmax_cross_entropy(one_hot_y, logits)
optimizer = tf.train.MomentumOptimizer(learning_rate=LR,
                                       momentum=Mom)
training = optimizer.minimize(cost)